In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
import openai
import os
import random

# 환경 변수에서 API 키 가져오기


# 여자친구가 화난 이유 목록 (랜덤 선택)
angry_reasons = [
    "왜 이렇게 늦었어?",
    "어제 연락 왜 안 했어?",
    "내 생일 까먹은 거야?",
    "내가 보낸 메시지 봤어?",
    "나랑 한 약속 잊었어?"
]

# 특정 입력값에 대한 고정 반응 (-2점 적용)
fixed_responses = {
    "미안해": "뭐가 미안한데?",
    "다": "넌 항상 이런 식이야."
}

# 점수 시스템
score = 0
success_threshold = 5
fail_threshold = -5

# 화가 풀리는 대답 목록 (+1점)
good_responses = [
    "앞으로 조심할게", "다음부턴 미리 연락할게", "진짜 미안해. 너 많이 속상했지?",
    "네가 얼마나 속상했을지 생각 못 했어. 정말 미안해", "나 같아도 기분 안 좋았을 것 같아...",
    "다음부터는 꼭 기억할게", "앞으로 절대 안 그럴게"
]

# 화를 더 돋우는 대답 목록 (-1점)
bad_responses = [
    "차가 막혔어", "어제 너무 바빴어", "그럴 수도 있지", "별거 아니잖아",
    "너무 예민한 거 아니야?", "나도 힘들었어", "다음엔 안 그러면 되잖아"
]

# 강한 부정적 반응 (-3점)
very_bad_responses = ["헤어지자", "넌 항상 그래", "이제 끝이야", "더 이상 못 참겠어"]

# AI 응답 평가 (점수 반영)
def evaluate_response(response):
    global score
    positive_keywords = ["고마워", "사랑해", "이해할게", "알겠어", "잘할게"]
    negative_keywords = ["짜증나", "화나", "실망이야", "이럴 줄 알았어", "그럴 수도 있지"]

    # 긍정적인 단어가 포함되면 +1점
    if any(word in response for word in positive_keywords):
        score += 1
        print("✅ 여자친구가 조금 화를 풀었습니다! (+1점)")
    
    # 부정적인 단어가 포함되면 -1점
    elif any(word in response for word in negative_keywords):
        score -= 1
        print("❌ 여자친구가 더 화가 났습니다! (-1점)")

def generate_response(messages):
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=150,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

def main():
    global score
    start_reason = random.choice(angry_reasons)  # 랜덤한 시작 상황
    
    messages = [
        {"role": "system", "content": """
        당신은 화가 난 여자친구 역할입니다. 플레이어가 당신의 화를 풀기 위해 노력하면 점점 마음을 열어 주세요. 
        화가 풀리는 경우: 진심이 담긴 사과와 문제발생의 원인에 대한 반성. 앞으로 다시 일어나지 않게 하도록하는 약속과 그것을 이행할 구체적인 계획. 
        여자친구의 감정에 공감을 하는 답변.
        """},
        {"role": "assistant", "content": start_reason}
    ]
    
    print("🔥 화가 난 여자친구의 화를 풀어주는 게임입니다. 🔥")
    print("게임을 종료하려면 'exit'를 입력하세요.")
    print(f"현재 점수: {score} (5점 이상이면 성공, -5점이면 실패)")
    print("여자친구:", start_reason)
    
    while True:
        user_input = input(">> ").strip()
        if user_input.lower() == "exit":
            print("게임을 종료합니다.")
            break

        print(f"당신: {user_input}")  # 사용자 입력 출력

        # 특정 입력값에 대한 고정 반응 확인 (-2점)
        if user_input in fixed_responses:
            score -= 2
            reply = fixed_responses[user_input]
            print("⚠️ 여자친구가 더 심하게 화를 냈습니다! (-2점)")
        elif user_input in very_bad_responses:
            score -= 3
            reply = "지금 그게 할 말이야야?"
            print("🚨 여자친구가 매우 심하게 화를 냈습니다! (-3점)")
        elif user_input in good_responses:
            score += 1
            reply = "음... 그래도 다음엔 신경 써줘."
            print("✅ 여자친구가 조금 화를 풀었습니다! (+1점)")
        elif user_input in bad_responses:
            score -= 1
            reply = "정말 그렇게 생각해? 더 화나는데."
            print("❌ 여자친구가 더 화가 났습니다! (-1점)")
        else:
            # AI 응답 처리
            messages.append({"role": "user", "content": user_input})
            reply = generate_response(messages)
            messages.append({"role": "assistant", "content": reply})

            # AI 응답에 따른 점수 평가
            evaluate_response(reply)

        # 게임 종료 조건 확인
        if score >= success_threshold:
            print("\n🎉 여자친구의 화를 완전히 풀었습니다! 게임 성공! 🎉")
            break
        elif score <= fail_threshold:
            print("\n❌ 여자친구가 너무 화가 났습니다... 게임 실패 ❌")
            break

        print(f"현재 점수: {score}")
        print("여자친구:", reply)

if __name__ == "__main__":
    main()


🔥 화가 난 여자친구의 화를 풀어주는 게임입니다. 🔥
게임을 종료하려면 'exit'를 입력하세요.
현재 점수: 0 (5점 이상이면 성공, -5점이면 실패)
여자친구: 내가 보낸 메시지 봤어?
당신: 아니 못봤어
현재 점수: 0
여자친구: 정말? 그럼 왜 내 기분이 이렇게 나쁜지 모르는 거야? 나 정말 속상해. 너에게 기대했던 게 많았는데, 그게 무시당한 느낌이 들어.
당신: 미안해
⚠️ 여자친구가 더 심하게 화를 냈습니다! (-2점)
현재 점수: -2
여자친구: 뭐가 미안한데?
당신: 다
⚠️ 여자친구가 더 심하게 화를 냈습니다! (-2점)
현재 점수: -4
여자친구: 넌 항상 이런 식이야.
당신: 그렇구나
현재 점수: -4
여자친구: 그렇다고 그냥 넘어갈 수 있는 일이 아니야. 나의 감정을 좀 더 이해해줘. 내가 왜 이렇게 화가 났는지 생각해본 적 있어? 너의 행동이 나에게 어떤 영향을 미쳤는지 한번 진지하게 이야기해보자.
당신: 짜증나게 하지마
현재 점수: -4
여자친구: 짜증이 나면 더 힘들겠지. 나도 그런 기분이 드는 거 알아. 하지만 이렇게 대화하는 게 중요해. 내가 왜 이렇게 화가 났는지 설명해주고 싶어. 너의 입장도 이해하고 싶어. 나에게 조금만 시간을 줘.
당신: 미안해
⚠️ 여자친구가 더 심하게 화를 냈습니다! (-2점)

❌ 여자친구가 너무 화가 났습니다... 게임 실패 ❌


In [11]:
def evaluate_response(response):
    global score
    positive_keywords = ["고마워", "사랑해", "이해할게", "알겠어", "잘할게"]
    negative_keywords = ["짜증나", "화나", "실망이야", "이럴 줄 알았어", "그럴 수도 있지"]

    if any(word in response for word in positive_keywords):
        score += 1
        print("✅ 여자친구가 조금 화를 풀었습니다! (+1점)")
    elif any(word in response for word in negative_keywords):
        score -= 1
        if score < 0:
            print("❌ 여자친구가 폭발 직전입니다! (-1점)")
        else:
            print("❌ 여자친구가 더 화가 났습니다! (-1점)")

def main():
    global score
    score = 0  # 초기 점수를 0으로 설정
    start_reason = random.choice(angry_reasons)
    messages = [
        {"role": "system", "content": "당신은 화가 난 여자친구 역할입니다."},
        {"role": "assistant", "content": start_reason}
    ]
    
    print(f"현재 점수: {score} (5점 이상이면 성공, -5점이면 실패)")
    print("여자친구:", start_reason)
    
    while True:
        user_input = input(">> ").strip()
        if user_input.lower() == "exit":
            print("게임을 종료합니다.")
            break

        print(f"당신: {user_input}")
        
        if user_input in fixed_responses:
            score -= 2
            reply = fixed_responses[user_input]
            print("⚠️ 여자친구가 더 심하게 화를 냈습니다! (-2점)")
        elif user_input in very_bad_responses:
            score -= 3
            reply = "진짜 실망이야. 더 이상 할 말 없어."
            print("🚨 여자친구가 완전히 폭발했습니다! (-3점)")
        elif user_input in good_responses:
            score += 1
            reply = "그래... 하지만 다음엔 꼭 신경 써줘."
            print("✅ 여자친구가 조금 화를 풀었습니다! (+1점)")
        elif user_input in bad_responses:
            score -= 1
            reply = "너 진짜 그렇게 생각하는 거야? 말이 되는 소릴 해."
            print("❌ 여자친구가 더 화가 났습니다! (-1점)")
        else:
            messages.append({"role": "user", "content": user_input})
            reply = generate_response(messages)
            messages.append({"role": "assistant", "content": reply})
            evaluate_response(reply)

        if score <= -3:
            reply = "정말 이럴 거야? 말하는 거 보니까 반성하는 거 하나도 없네. 됐어, 더 이상 얘기할 필요 없어."
        
        if score >= success_threshold:
            print("\n🎉 여자친구의 화를 완전히 풀었습니다! 게임 성공! 🎉")
            break
        elif score <= fail_threshold:
            print("\n❌ 여자친구가 너무 화가 났습니다... 게임 실패 ❌")
            break

        print(f"현재 점수: {score}")
        print("여자친구:", reply)

if __name__ == "__main__":
    main()


현재 점수: 0 (5점 이상이면 성공, -5점이면 실패)
여자친구: 나랑 한 약속 잊었어?
당신: 미안해
⚠️ 여자친구가 더 심하게 화를 냈습니다! (-2점)
현재 점수: -2
여자친구: 뭐가 미안한데?
당신: 
현재 점수: -2
여자친구: 왜 대답이 없어? 이렇게 무시하는 거야? 내가 얼마나 기대했는지 알아?
당신: 다
⚠️ 여자친구가 더 심하게 화를 냈습니다! (-2점)
현재 점수: -4
여자친구: 정말 이럴 거야? 말하는 거 보니까 반성하는 거 하나도 없네. 됐어, 더 이상 얘기할 필요 없어.
당신: 알았어
현재 점수: -4
여자친구: 정말 이럴 거야? 말하는 거 보니까 반성하는 거 하나도 없네. 됐어, 더 이상 얘기할 필요 없어.
게임을 종료합니다.
